In [104]:
# Impport Required packages
import pandas as pd
import numpy as np
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
en = spacy.load('en_core_web_sm')
en.add_pipe('spacytextblob')
import deplacy
import stanza
stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize, sentiment')
from statistics import mean
import xml.etree.ElementTree as ET

2022-01-07 14:55:43 INFO: Downloading default packages for language: en (English)...
2022-01-07 14:55:44 INFO: File exists: C:\Users\vibkr\stanza_resources\en\default.zip.
2022-01-07 14:55:47 INFO: Finished downloading models and saved to C:\Users\vibkr\stanza_resources.
2022-01-07 14:55:47 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| sentiment | sstplus  |

2022-01-07 14:55:47 INFO: Use device: cpu
2022-01-07 14:55:47 INFO: Loading: tokenize
2022-01-07 14:55:47 INFO: Loading: sentiment
2022-01-07 14:55:47 INFO: Done loading processors!


In [2]:
# Read raw reviews
# df = pd.read_csv("scraped_output.csv")
df = pd.read_csv("scraped_output.csv")
df.dropna(inplace=True)
df.head(3)

,Unnamed: 0,Review,Date,Stars,Restaurant
0,0,"(Translated by Google) Good hamburgers, delici...",5 days ago,4 stars,TFDB
3,3,"Burger was ok, not fantastic. Place was extre...",2 weeks ago,3 stars,TFDB
4,4,"One of the better burgers I've had, the bread ...",2 weeks ago,5 stars,TFDB


In [42]:
def get_feature_descriptors(text):
    doc = en(text)
    # deplacy.render(doc)
    words_seen = set()
    fd_dict = dict()
    for sent in doc.sents:
        for word in sent:
            if word.pos_ == "NOUN":
                temp_noun = word.text;
                fd_dict[word.text] = [] # Empty list
            if word.pos_ in ["ADV", "ADJ", "VERB"]:
                try:
                    fd_dict[temp_noun].append(word.text)
                except:
                    pass;
    return [fd_dict]

In [43]:
txt = """The food was really good, and my mom thinks it is amazing"""
get_feature_descriptors(txt)


[{'food': ['really', 'good'], 'mom': ['thinks', 'amazing']}]

In [44]:
# Apply function to get feature-descriptors
df["feature_descriptors"] = df["Review"].apply(lambda x:get_feature_descriptors(x))

In [45]:
df.head(3)

,Unnamed: 0,Review,Date,Stars,Restaurant,feature_descriptors
0,0,"(Translated by Google) Good hamburgers, delici...",5 days ago,4 stars,TFDB,"[{'hamburgers': ['delicious', 'great'], 'cockt..."
3,3,"Burger was ok, not fantastic. Place was extre...",2 weeks ago,3 stars,TFDB,"[{'night': [], 'room': [], 'temperature': [], ..."
4,4,"One of the better burgers I've had, the bread ...",2 weeks ago,5 stars,TFDB,"[{'burgers': ['had'], 'bread': ['soft', 'well'..."


In [76]:
def get_review_level_sentiment(feat):
    feat = feat[0]
    comb_dict = dict()
    for key in feat.keys():
        arr = feat[key]
        if key in comb_dict:
            for word in arr:
                doc = nlp(word)
                for i, sentence in enumerate(doc.sentences):
                    # print(word, sentence.sentiment)
                    comb_dict[key].append(sentence.sentiment)
        else:
            comb_dict[key] = []
            for word in arr:
                doc = nlp(word)
                for i, sentence in enumerate(doc.sentences):
                    # print(word, sentence.sentiment)
                    comb_dict[key].append(sentence.sentiment-1)
    return [comb_dict]


        


In [70]:
df.iloc[5,5]

[{'burgers': ['reminds', 'only'],
  'thing': ['missing', 'perfect'],
  'weather': ['Fantastic'],
  'char': ['patty'],
  'onion': [],
  'rings': ['nicely', 'done']}]

In [75]:
df["sentiment_feat"] = df["feature_descriptors"].apply(lambda x:get_review_level_sentiment(x))

delicious 2
great 2
Original 2
had 1
soft 1
well 2
toasted 1
is 1
thick 1
flavorful 2
popped 1
was 1
wonderful 2
had 1
had 1
nice 2
had 1
smaller 1
come 1
affordable 2
come 1
tasty 2
are 1
good 2
reminds 1
only 1
missing 1
perfect 2
Fantastic 2
patty 1
nicely 2
done 1
maybe 1
full 2
available 1
seared 1
well 2
good 2
Overall 1
come 1
again 1
quite 1
possibly 1
has 1
best 2
tried 1
were 1
superb 2
juicy 2
well 2
seasoned 1
tasty 2
is 1
placed 1
neatly 1
come 1
back 1
is 1
still 1
better 2
offered 1
really 1
great 2
Enjoyed 2
juicy 2
fluffy 1
have 1
cheap 0
as 1
well 2
good 2
nicely 2
scrambled 1
mushy 0
sure 1
supposed 1
be 1
weird 0
hard 1
enough 1
too 1
soft 1
's 1
much 1
more 1
juicier 1
go 1
rare 2
here 1
excellent 2
actually 1
ask 1
honest 2
great 2
Took 1
had 1
quite 1
large 1
really 1
filling 2
so 1
here 1
sublime 2
had 1
patty 1
is 1
slightly 1
crispy 1
touching 2
adds 1
delectable 2
quite 1
long 1
even 1
waiting 1
accept 1
was 1
still 1
pretty 2
juicy 2
Fresh 2
add 2
was 1
own 

In [77]:
df.head(3)

,Unnamed: 0,Review,Date,Stars,Restaurant,feature_descriptors,sentiment_feat
0,0,"(Translated by Google) Good hamburgers, delici...",5 days ago,4 stars,TFDB,"[{'hamburgers': ['delicious', 'great'], 'cockt...","[{'hamburgers': [1, 1], 'cocktails': [1]}]"
3,3,"Burger was ok, not fantastic. Place was extre...",2 weeks ago,3 stars,TFDB,"[{'night': [], 'room': [], 'temperature': [], ...","[{'night': [], 'room': [], 'temperature': [], ..."
4,4,"One of the better burgers I've had, the bread ...",2 weeks ago,5 stars,TFDB,"[{'burgers': ['had'], 'bread': ['soft', 'well'...","[{'burgers': [0], 'bread': [0, 1, 0], 'meat': ..."


In [103]:
# Get avg sentiment for each feature at restaurant level

restaurants = df["Restaurant"].unique()
restaurants = list(restaurants)

avg_sent_list = []

for rest in restaurants:
    tdf = df[df["Restaurant"] == rest]

    rmap = dict()

    ser = tdf["sentiment_feat"].tolist()
    ser = [item for sublist in ser for item in sublist]
    # print(ser)
    
    for d in ser:
        rmap.update(d)
    
    for key in rmap.keys():
        if len(rmap[key]) > 0:
            m = mean(rmap[key])
            rmap[key] = m
    # print(rmap)
    avg_sent_list.append((rest, rmap))


In [96]:
# further process to categorise



1